# Benchmark calculcations for January and June proposals

This notebook uses the `baseline_model` to calculate the expected emissions between 2027 and 2050 based on two proposed BEPS timelines:
    
- The January proposal, which uses 2027 as the first benchmark year
- The June proposal, which uses 2030 as the first benchmark year
- NB: January and June refer to the months that OSE proposed each timeline

The notebook produces four CSVs:
    
- two CSVs showing the expected emissions (in kg) for each building subject to the policy for each year between 2027 and 2050, one for the January timeline and one for the June timeline
- two CSVs showing the expected total annual emissions (in kg) in 2027-2050, one for the January timeline and one for the June timeline

**A Note on the Model History:**

The original model was built in Google Sheets by a team at [RMI](https://rmi.org/), who kindly provided it to 350 Seattle. The spreadsheet format was difficult to expand and contained several errors, so it has been rewritten as a Python class by Isaac Cowhey. This format makes catching errors and expanding the model easier.

**Source Data:**

This notebook uses the publicly available building data from 2019, with calculations to create new fields, namely mapping the OSE building types to policy building types and the percentage of building area dedicated to each use type. This can be found in the data/input_data/Data cleaning.ipynb notebook. 


**Assumptions**:

- Buildings will produce the max amount of emissions, **unless** their predicted emissions is lower than the target GHGI, in which case they will produce that amount.
- Predicted emissions are calculated based off of the emissions predictions found in [the emissions data input file](../data/input_data/energy_emissions.csv). 

**Methods**:

For each building, in each year (2027-2050), we need to calculate the following (column references given refer to the 2027 calculations in [the original RMI spreadsheet](https://docs.google.com/spreadsheets/d/175uipAHHQHGelq7i1n9sKWQXNQi-B1IJiF6-XQRVnE8/edit#gid=1811888818):

- `city_ghgi_target`: The GHGI (greenhouse gas emissions intensity) target, as calculated by the sum of: `city's target GHGI for a use type * percent of GFA for that use type`, for each of the three given use types (col A). If there is no target for any of the use types, this is `NaN`. If a building is multi-use, and some of the building's uses have a compliance threshold but others don't, the expected GHGI is calculated as: `baseline GHGI * percent of GFA for use types not yet subject to compliance threshold + GHGI target for use types subject to compliance threshold (see above calculation)`.
    - Example: A building is 50% retail and 50% multifamily housing. The target for retail in 2033 is 1.03 and there is no target for multifamily housing. This building would have a GHGI of 4.0 in 2033 if no changes were made to it. We estimate the GHGHI target as `(0.5 * 1.03) + (0.5 * 4.0) = 2.515`. 
    - NB: These numbers will be different than the RMI calculations. The RMI model mistakenly used a GHGI of zero for parts of buildings that are not yet subject to BEPS. So in the example above, RMI's model would list the GHGI target as`0.5 * 1.03 + 0.5 * 0 = 0.515`, which leaves out half the building. Then the target GHGI would actually go **up** when the multifamily part of the building became subject to BEPS! This model corrects that error.
- `expected_baseline`: The expected emissions if nothing is changed about the building, as calculated by the sum of: `total energy of a given type * energy emissions factor for energy type` for the three energy types (col C)
- `expected_baseline_ghgi`: The expected GHGI if nothing is changed about the building (col B), as calculated by the `expected_baseline / total GFA`
- `compliant_ghgi`: The expected GHGI if the building is compliant with BEPS, as defined by (col H):
    - if the BEPS GHGI target is lower than the expected baseline GHGI, use the BEPS GHGI target
    - if the expected baseline GHGI is lower than the BEPS GHGI target, use the expected baseline GHGI
- `compliant_emissions`: The expected emissions if the building is compliant with BEPS, as defined by the `compliant GHGI * total GFA` (col J)
- `compliance_status`: Whether or not the building is compliant (col K):
    - yes: the baseline GHGI is lower than the expected compliant GHGI for this year
    - no: the baseline GHGI is higher than the expected compliant GHGI for this year
    - no requirement yet: the building doesn't have a compliance requirement for this year
- `compliance_fees`: Noncompliance fees. For years where buildings will be taxed for being noncompliant, this is `compliance fee per sq ft * total GFA`

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from baseline_model import BaselineBEPSModel

In [3]:
pd.options.mode.chained_assignment = None

In [4]:
JAN_TARGETS_PATH = '../data/input_data/jan_proposal_emissions_targets.csv'
JUNE_TARGETS_PATH = '../data/input_data/june_proposal_emissions_targets.csv'
EMISSIONS_PATH = '../data/input_data/energy_emissions.csv'
BUILDING_DATA_PATH = '../data/input_data/Data cleaning/cleaned_building_data_with_policy_gfa.csv'

JAN_FINE_YEARS = [2027, 2030, 2035, 2040, 2045, 2050]
JUNE_FINE_YEARS = [2030, 2035, 2040, 2045, 2050]
FINE_PER_SQ_FT = 2.5

## January Proposal

In [5]:
jan_model = BaselineBEPSModel(EMISSIONS_PATH, JAN_TARGETS_PATH, BUILDING_DATA_PATH, JAN_FINE_YEARS, FINE_PER_SQ_FT)

In [6]:
jan_model.calculate_baseline_model(2027,2050)

Model calculations complete. Access the model dataframe as model_name.scenario_results


In [7]:
jan_model.scenario_results.head()

,OSEBuildingID,BuildingName,Total GFA for Policy,sq_ft_classification,LargestPropertyUseType OSE,SecondLargestPropertyUseType OSE,ThirdLargestPropertyUseType OSE,year,expected_baseline,expected_baseline_ghgi,city_ghgi_target,compliant_ghgi,compliant_emissions,compliance_status,compliance_fees
0,1,MAYFLOWER PARK HOTEL,88434.0,C,Hotel,NaN,NaN,2027.0,2.705014e+05,3.058794,3.058794,3.058794,2.705014e+05,No,221085.0
1,2,PARAMOUNT HOTEL,88502.0,C,Hotel,NaN,Restaurant,2027.0,2.843616e+05,3.213052,3.213052,3.213052,2.843616e+05,No,221255.0
2,3,WESTIN HOTEL (Parent Building),756493.0,A,Hotel,NaN,Recreation,2027.0,2.193901e+06,2.900095,2.220000,2.220000,1.679414e+06,No,1891232.5
3,5,HOTEL MAX,61320.0,C,Hotel,NaN,NaN,2027.0,2.934861e+05,4.786141,4.786141,4.786141,2.934861e+05,No,153300.0
4,8,WARWICK SEATTLE HOTEL,123445.0,B,Hotel,NaN,Recreation,2027.0,5.827520e+05,4.720742,2.220000,2.220000,2.740479e+05,No,308612.5


## Summary stats for January Proposal

In [8]:
jan_model.get_total_emissions_by_year()

Emissions by year calculations complete. Access the annual emissions dataframe as model_name.emissions_by_year


In [9]:
jan_model.emissions_by_year.head()

,compliant_emissions
year,
2027.0,3.367781e+08
2028.0,3.239650e+08
2029.0,3.072631e+08
2030.0,2.779739e+08
2031.0,2.457496e+08


## Calculate percent of CO2 reduction by 2040

For an aggressive climate policy, 350 Seattle would prefer that emissions are cut by 90% by 2040. 

In [10]:
jan_model.get_percent_emissions_reduction_by_given_year(2040)

0.868468817112779

In [11]:
jan_model.emissions_by_year.sum()

compliant_emissions    3.227114e+09
dtype: float64

The January proposal results in 3,220,000 metric tons (3.22 * 10^9 kg) of carbon emitted between 2027 and 2050.

By 2040, the January proposal will have reduced emissions from large buildings in Seattle by 86.8%.

## June proposal

In [12]:
june_model = BaselineBEPSModel(EMISSIONS_PATH, JUNE_TARGETS_PATH, BUILDING_DATA_PATH, JUNE_FINE_YEARS, FINE_PER_SQ_FT)

In [13]:
june_model.calculate_baseline_model(2027,2050)

Model calculations complete. Access the model dataframe as model_name.scenario_results


In [14]:
june_model.scenario_results.head()

,OSEBuildingID,BuildingName,Total GFA for Policy,sq_ft_classification,LargestPropertyUseType OSE,SecondLargestPropertyUseType OSE,ThirdLargestPropertyUseType OSE,year,expected_baseline,expected_baseline_ghgi,city_ghgi_target,compliant_ghgi,compliant_emissions,compliance_status,compliance_fees
0,1,MAYFLOWER PARK HOTEL,88434.0,C,Hotel,NaN,NaN,2027.0,2.705014e+05,3.058794,3.058794,3.058794,2.705014e+05,No,0.0
1,2,PARAMOUNT HOTEL,88502.0,C,Hotel,NaN,Restaurant,2027.0,2.843616e+05,3.213052,3.213052,3.213052,2.843616e+05,No,0.0
2,3,WESTIN HOTEL (Parent Building),756493.0,A,Hotel,NaN,Recreation,2027.0,2.193901e+06,2.900095,2.900095,2.900095,2.193901e+06,No,0.0
3,5,HOTEL MAX,61320.0,C,Hotel,NaN,NaN,2027.0,2.934861e+05,4.786141,4.786141,4.786141,2.934861e+05,No,0.0
4,8,WARWICK SEATTLE HOTEL,123445.0,B,Hotel,NaN,Recreation,2027.0,5.827520e+05,4.720742,4.720742,4.720742,5.827520e+05,No,0.0


## Summary stats for June

In [15]:
june_model.get_total_emissions_by_year()

Emissions by year calculations complete. Access the annual emissions dataframe as model_name.emissions_by_year


In [16]:
june_model.emissions_by_year.head()

,compliant_emissions
year,
2027.0,9.310595e+08
2028.0,9.310595e+08
2029.0,9.310595e+08
2030.0,8.970770e+08
2031.0,3.434390e+08


In [17]:
june_model.get_percent_emissions_reduction_by_given_year(2040)

0.8208085863032639

In [18]:
june_model.emissions_by_year.sum()

compliant_emissions    6.613525e+09
dtype: float64

The June proposal results in 6,610,000 metric tons (6.61 * 10**9 kgs) of carbon emitted between 2027 and 2050.

By 2040, the June proposal will have reduced emissions from large buildings in Seattle by 82%.


In [19]:
june_model.scenario_results.to_csv('../data/output_data/cleaned_public_data_june_scenario.csv')
june_model.emissions_by_year.to_csv('../data/output_data/cleaned_public_data_june_scenario_by_year.csv')

jan_model.scenario_results.to_csv('../data/output_data/cleaned_public_data_jan_scenario.csv')
jan_model.emissions_by_year.to_csv('../data/output_data/cleaned_public_data_jan_scenario_by_year.csv')

## Comparing January and June

In [ ]:
june_model.emissions_by_year.sum()

In [ ]:
jan_model.emissions_by_year.sum()

In [ ]:
sns.lineplot(data=june_model.emissions_by_year)
plt.show()

In [ ]:
sns.lineplot(data=jan_model.emissions_by_year)
plt.show()

That's twice as much emissions in June vs January.

However, the steep drop in the June proposal isn't completely out of whack. What's going on is that the electricity emissions factor drops by 50% in 2030. So that's a huge reduction. Then the first targets go into effect in 2031. Between those two things, 